# Accidents Corporels de la Circulation Routière 2016
## Utilisation de Shapash

Sources:

Données: data.gouv.fr

https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2019/


Shapash: 

https://pub.towardsai.net/shapash-making-ml-models-understandable-by-everyone-8f96ad469eb3

https://github.com/MAIF/shapash

## Data preparation

In [1]:
### Les packages  
import time
import numpy as np 
import pandas as pd
import string
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, HTML
import collections
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from functools import reduce
from sklearn.metrics import confusion_matrix

### Les options générales
pd.set_option("display.max_columns", 99) # permet de voir toutes les colonnes

In [2]:
### Chargement des données

#Impossible de charger le fichier caractéristiques avec l'ensemble des colonnes
#d1 Caractéristiques de l'accident
d1 = pd.read_csv("caracteristiques_2016_col_reduit.csv" #59 432
                 ,sep=','
                 ,header='infer'
                 ,encoding='utf-8')

#d2 Lieux de l'accident
d2 = pd.read_csv("lieux_2016.csv" #59 432
                 ,sep=','
                 ,header='infer'
                 ,encoding='utf-8')

#d3 Véhicules accidentés
d3 = pd.read_csv("vehicules_2016.csv" #101 924
                 ,sep=','
                 ,header='infer'
                 ,encoding='utf-8')

#d4 Usagers
d4 = pd.read_csv("usagers_2016.csv" #133 422
                 ,sep=','
                 ,header='infer'
                 ,encoding='utf-8')

/home/11942h/.conda/envs/FBE_py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
### Recodages, renommages et constitution de la table de travail

d1['heure'] = d1['hrmn'].apply(lambda x: '{0:0>4}'.format(x)).str.slice(stop=2).astype(int) # On ne garde que l'heure de l'accident
d1['col'].fillna(6, inplace=True)
d1.rename(columns={'lum':'lumiere'}, inplace=True)
d1.rename(columns={'agg':'agglomeration'}, inplace=True)
d1.rename(columns={'int':'intersection'}, inplace=True)
d1.rename(columns={'atm':'condition_atmospherique'}, inplace=True)
d1.rename(columns={'col':'collision'}, inplace=True)

d2.rename(columns={'nbv':'nombre_voies'}, inplace=True)
d2.rename(columns={'catr':'categorie_route'}, inplace=True)
d2.rename(columns={'circ':'circulation'}, inplace=True)
d2.rename(columns={'prof':'profil'}, inplace=True)
d2.rename(columns={'surf':'etat_surface'}, inplace=True)
d2.rename(columns={'infra':'infrastructure'}, inplace=True)
d2.rename(columns={'infra':'infrastructure'}, inplace=True)
d2.rename(columns={'vosp':'voie_reservee'}, inplace=True)
d2.rename(columns={'plan':'trace'}, inplace=True)
d2['nombre_voies'] = np.where(d2['nombre_voies'] > 8, 8, d2['nombre_voies']) # On limite de nombre de voies à 8 et plus

d3.rename(columns={'obs':'obstacle'}, inplace=True)
d3['catv'] = np.where(d3['catv'] == 35, 36, d3['catv']) # Regroupement quads
d3.rename(columns={'catv':'categorie_veh'}, inplace=True)
d3.rename(columns={'obsm':'obstacle_mobile'}, inplace=True)

d3b = d3.groupby(['Num_Acc']).size().reset_index(name='nb_veh')
d3b.rename(columns={'nb_veh':'nombre_vehicules'}, inplace=True)
d3b['nombre_vehicules'] = np.where(d3b['nombre_vehicules'] > 8, 8, d3b['nombre_vehicules']) # On limite à 8 véhicules concernés et plus

d4.rename(columns={'grav':'gravite'}, inplace=True)
d4.rename(columns={'catu':'usager'}, inplace=True)

d4b = d4.groupby(['Num_Acc']).size().reset_index(name='nb_usagers')
d4c = d4[(d4.gravite == 2)].groupby(['Num_Acc']).size().reset_index(name='nb_tues')
d4b.rename(columns={'nb_usagers':'nombre_usagers'}, inplace=True)
d4b['nombre_usagers'] = np.where(d4b['nombre_usagers'] >8, 8, d4b['nombre_usagers']) # On limite à 8 usagers et plus


# Regroupements - Table de travail à l'usager
d10 = pd.merge(d3, d4, on=['Num_Acc', 'num_veh'], how='inner')
dfs = [d10, d1, d2, d3b, d4b, d4c]
d11 = reduce(lambda  left,right: pd.merge(left, right, on=['Num_Acc'], how='left'), dfs).fillna(0)

# Recodage de la variable endogène
y = d4['gravite'] == 2


In [4]:
### Dictionnaire
ta = {'col': 'lumiere',
       'mapping': pd.Series(data=[1, 2, 3, 4, 5, 0],
                            index=['Plein jour', 'Crépuscule ou aube', 'Nuit sans éclairage public',
                                   'Nuit avec éclairage public non allumé', 'Nuit avec éclairage public allumé',
                                   'Non renseigné']),
       'data_type': 'object'}

tb = {'col': 'agglomeration',
       'mapping': pd.Series(data=[1, 2, 0],
                            index=['Hors agglomération', 'En agglomératio', 'Non renseigné']),
       'data_type': 'object'}

tc = {'col': 'intersection',
        'mapping': pd.Series(data=[1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
                             index=['Hors intersection', 'Intersection en X', 'Intersection en T', 'Intersection en Y',
                                   'Intersection à plus de 4 branches', 'Giratoire', 'Place', 'Passage à niveau',
                                   'Autre intersection', 'Non renseigné']),
       'data_type': 'object'}

td = {'col': 'condition_atmospherique',
        'mapping': pd.Series(data=[1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
                             index=['Normale', 'Pluie légère', 'Pluie forte', 'Neige - grêle', 'Brouillard - fumée',
                                    'Vent fort - tempête', 'Temps éblouissant', 'Temps couvert', 'Autre', 'Non renseigné']),
       'data_type': 'object'}

te = {'col': 'collision',
        'mapping': pd.Series(data=[1, 2, 3, 4, 5, 6, 7,8],
                             index=['2 véhicules - frontale', '2 véhicules – arrière', '2 véhicules – coté', '3 véh et + – en chaîne',
                                    '3 véh et + - collisions mult', 'Autre collision', 'Sans collision', 'Passage à niveau']),
       'data_type': 'object'}

tf = {'col': 'categorie_route',
        'mapping': pd.Series(data=[1, 2, 3, 4, 5, 6, 7, 0],
                             index=['Autoroute', 'Route Nationale', 'Route Départementale', 'Voie Communale', 'Hors réseau public',
                                    'Parc de stationnement', 'autre', 'Non renseigné']),
       'data_type': 'object'}

tg = {'col': 'circulation',
        'mapping': pd.Series(data=[1, 2, 3, 4, 0],
                             index=['A sens unique', 'Bidirectionnelle', 'A chaussées séparées', 'Voies affectation variable',
                                    'Non renseigné']),
       'data_type': 'object'}

th = {'col': 'profil',
        'mapping': pd.Series(data=[1, 2, 3, 4, 0],
                             index=['Plat', 'Pente', 'Sommet de côte', 'Bas de côte', 'Non renseigné']),
       'data_type': 'object'}

ti = {'col': 'infrastructure',
        'mapping': pd.Series(data=[1, 2, 3, 4, 5, 6, 7, 8, 0],
                             index=['Rien', 'Souterrain - tunnel', 'Pont - autopont', 'Bretelle ', 'Voie ferrée', 
                                    'Carrefour aménagé', 'Zone piétonne', 'Zone de péage', 'Non renseigné']),
       'data_type': 'object'}

tj = {'col': 'voie_reservee',
        'mapping': pd.Series(data=[1, 2, 3, 0],
                             index=['Piste cyclable', 'Banque cyclable', 'Voie réservée', 'Non renseigné']),
       'data_type': 'object'}

tk = {'col': 'trace',
        'mapping': pd.Series(data=[1, 2, 3, 4, 0],
                             index=['Partie rectiligne', 'En courbe à gauche', 'En courbe à droite', 'En S', 'Non renseigné']),
       'data_type': 'object'}

tl = {'col': 'obstacle',
        'mapping': pd.Series(data=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10 , 11, 12, 13, 14, 15, 16, 0],
                             index=['Véhicule en stationnement', 'Arbre', 'Glissière métallique', 'Glissière béton', 'Autre glissière'
                                    , 'Bâtiment, mur, pile', 'Support de signalisation', 'Poteau', 'Mobilier urbain', 'Parapet'
                                    , 'Ilot, refuge, borne haute', 'Bordure de trottoir', 'Fossé, talus, paroi rocheuse'
                                    , 'Autre obstacle fixe sur chaussée', 'Autre obstacle fixe sur trottoir ou accotement'
                                    , 'Sortie de chaussée sans obstacle', 'Non renseigné']),
       'data_type': 'object'}

tm = {'col': 'obstacle_mobile',
        'mapping': pd.Series(data=[1, 2, 3, 4, 5, 6, 0],
                             index=['Piéton', 'Véhicule', 'Véhicule sur rail', 'Animal domestique' ,'Animal sauvage', 'Autre'
                                    , 'Non renseigné']),
       'data_type': 'object'}

tn = {'col': 'choc',
        'mapping': pd.Series(data=[1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
                             index=['Avant', 'Avant droit', 'Avant gauche', 'Arrière', 'Arrière droit', 'Arrière gauche'
                                    , 'Côté droit', 'Côté gauche', 'Chocs multiples (tonneaux)', 'Non renseigné']),
       'data_type': 'object'}

to = {'col': 'usager',
        'mapping': pd.Series(data=[1, 2, 3, 4, 0],
                             index=['Conducteur', 'Passagé', 'Piéton', 'Piéton en roller ou trotinette', 'Non renseigné']),
       'data_type': 'object'}

# Liste des transformations
encoder = [ta, tb, tc, td, te, tf, tg, th, ti, tj, tk, tl, tm, tn, to]

In [5]:
### Xtrain & Xtest
X = d11[['mois', 'jour', 'heure', 'nombre_voies', 'nombre_vehicules', 'nombre_usagers'
         ,'lumiere','agglomeration', 'intersection', 'condition_atmospherique', 'collision'
         , 'categorie_route', 'circulation', 'profil', 'infrastructure', 'voie_reservee'
         , 'trace',  'obstacle', 'obstacle_mobile', 'choc', 'usager']]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.75, random_state=1)

## Modelisation

In [6]:
### Modélisation

model = lgb.LGBMClassifier(max_depth=-1,
                               subsamble=0.7,
                               reg_lambda=2,
                               reg_alpha=0,
                               objective='binary',
                               num_leaves=8,
                               n_estimators=600,
                               learning_rate=0.03,
                               is_unbalance=True,
                               colsample_bytree=0.7,
                               boosting_type='gbdt',
                               random_state=314, silent=True, metric='None', n_jobs=4)
model.fit(Xtrain, ytrain)

ypred = model.predict(Xtest)

cm = confusion_matrix(ytest, ypred)
print(cm), np.trace(cm) / np.sum(cm)

[LightGBM] [Warning] Unknown parameter: subsamble
[[24142  8316]
 [  178   720]]


(None, 0.7453531598513011)

## Shapash

In [8]:
### Shapash

from shapash.explainer.smart_explainer import SmartExplainer

y_pred = pd.DataFrame(pd.Series(ypred).astype(int), index=Xtest.index, columns=['ypred']).fillna(0)

xpl = SmartExplainer()

xpl.compile(
    x = Xtest,
    preprocessing = encoder,
    model = model,
    y_pred = y_pred.ypred
)

Backend: Shap TreeExplainer


/home/11942h/.conda/envs/FBE_py38/lib/python3.8/site-packages/shap/explainers/_tree.py:300: UserWarning:

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray



In [ ]:
app = xpl.run_app(port=8051)

# Après exécution, à titre d'exemple sur le serveur utilisé dans cas: INFO:root:Your Shapash application run on http://slhdg001:8051/


INFO:werkzeug:172.16.87.97 - - [14/Apr/2021 12:02:48] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:172.16.87.97 - - [14/Apr/2021 12:02:49] "POST /_dash-update-component HTTP/1.1" 200 -
